## Import Library

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from imblearn.over_sampling import SMOTE

from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix, roc_auc_score, f1_score

## Load dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Materi Day 21/dataset/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [ ]:
df['OnlineBackup']

0       Yes
1        No
2       Yes
3        No
4        No
       ... 
7038     No
7039    Yes
7040     No
7041     No
7042     No
Name: OnlineBackup, Length: 7043, dtype: object

## Data Preprocessing

In [ ]:
# checking duplicate data
df.duplicated().sum()

0

In [ ]:
# change TotalCharges datatype because its data type must be float not object
df['TotalCharges'] = df['TotalCharges'].astype(float)

ValueError: ignored

In [ ]:
#apparently some things missing from the column. we will check it

df[df['TotalCharges']==' ']

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
488,4472-LVYGI,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,,No
753,3115-CZMZD,Male,0,No,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,,No
936,5709-LVOEQ,Female,0,Yes,Yes,0,Yes,No,DSL,Yes,...,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,,No
1082,4367-NUYAO,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,,No
1340,1371-DWPAZ,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,,No
3331,7644-OMVMY,Male,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.85,,No
3826,3213-VVOLG,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,,No
4380,2520-SGTTA,Female,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,,No
5218,2923-ARZLG,Male,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,,No
6670,4075-WKNIU,Female,0,Yes,Yes,0,Yes,Yes,DSL,No,...,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,,No


We have 11 rows that have no values in TotalCharges column. We can drop the rows because the percentage of missing values is just 0.1%

In [ ]:
df = df.drop([488, 753, 936, 1082, 1340, 3331, 3826, 4380, 5218, 6670, 6754])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7032 non-null   object 
 1   gender            7032 non-null   object 
 2   SeniorCitizen     7032 non-null   int64  
 3   Partner           7032 non-null   object 
 4   Dependents        7032 non-null   object 
 5   tenure            7032 non-null   int64  
 6   PhoneService      7032 non-null   object 
 7   MultipleLines     7032 non-null   object 
 8   InternetService   7032 non-null   object 
 9   OnlineSecurity    7032 non-null   object 
 10  OnlineBackup      7032 non-null   object 
 11  DeviceProtection  7032 non-null   object 
 12  TechSupport       7032 non-null   object 
 13  StreamingTV       7032 non-null   object 
 14  StreamingMovies   7032 non-null   object 
 15  Contract          7032 non-null   object 
 16  PaperlessBilling  7032 non-null   object 


In [ ]:
# we can try to change the datatype again
df['TotalCharges'] = df['TotalCharges'].astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7032 non-null   object 
 1   gender            7032 non-null   object 
 2   SeniorCitizen     7032 non-null   int64  
 3   Partner           7032 non-null   object 
 4   Dependents        7032 non-null   object 
 5   tenure            7032 non-null   int64  
 6   PhoneService      7032 non-null   object 
 7   MultipleLines     7032 non-null   object 
 8   InternetService   7032 non-null   object 
 9   OnlineSecurity    7032 non-null   object 
 10  OnlineBackup      7032 non-null   object 
 11  DeviceProtection  7032 non-null   object 
 12  TechSupport       7032 non-null   object 
 13  StreamingTV       7032 non-null   object 
 14  StreamingMovies   7032 non-null   object 
 15  Contract          7032 non-null   object 
 16  PaperlessBilling  7032 non-null   object 


In [ ]:
# then we need to encode the Yes/No columns and gender column
def maping(x):
  if x == 'No':
    return 0
  else:
    return 1

df['Partners'] = df['Partner'].apply(maping)
df['Dependentss'] = df['Dependents'].apply(maping)
df['Phone Service'] = df['PhoneService'].apply(maping)
df['Online Security'] = df['OnlineSecurity'].apply(maping)
df['Online Backup'] = df['OnlineBackup'].apply(maping)
df['Device Protection'] = df['DeviceProtection'].apply(maping)
df['Tech Support'] = df['TechSupport'].apply(maping)
df['Streaming TV'] = df['StreamingTV'].apply(maping)
df['Streaming Movies'] = df['StreamingMovies'].apply(maping)
df['Paperless Billing'] = df['PaperlessBilling'].apply(maping)
df['Churns'] = df['Churn'].apply(maping)

# encode gender column
df['gender'] = df['gender'].replace('Female', 0).replace('Male', 1)

In [ ]:
# we can drop the unused columns

df.drop(['Partner', 'Dependents', 'PhoneService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'Churn'], axis = 'columns', inplace = True)
df.head()

,customerID,gender,SeniorCitizen,tenure,MultipleLines,InternetService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,...,Dependentss,Phone Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Paperless Billing,Churns
0,7590-VHVEG,0,0,1,No phone service,DSL,Month-to-month,Electronic check,29.85,29.85,...,0,0,0,1,0,0,0,0,1,0
1,5575-GNVDE,1,0,34,No,DSL,One year,Mailed check,56.95,1889.50,...,0,1,1,0,1,0,0,0,0,0
2,3668-QPYBK,1,0,2,No,DSL,Month-to-month,Mailed check,53.85,108.15,...,0,1,1,1,0,0,0,0,1,1
3,7795-CFOCW,1,0,45,No phone service,DSL,One year,Bank transfer (automatic),42.30,1840.75,...,0,0,1,0,1,1,0,0,0,0
4,9237-HQITU,0,0,2,No,Fiber optic,Month-to-month,Electronic check,70.70,151.65,...,0,1,0,0,0,0,0,0,1,1


In [ ]:
# we need to encode the categorical feature. in this dataset, I will use Frequency Encoding

freq_ml = df['MultipleLines'].value_counts().reset_index()
freq_ml.rename(columns={'index': 'MultipleLines', 'MultipleLines': 'freq_multiple_line'}, inplace = True)
freq_ml['pct_multiple_line'] = round((freq_ml['freq_multiple_line']/freq_ml['freq_multiple_line'].sum())*100,2)
freq_ml

,MultipleLines,freq_multiple_line,pct_multiple_line
0,No,3385,48.14
1,Yes,2967,42.19
2,No phone service,680,9.67


In [ ]:
df = df.merge(freq_ml[['MultipleLines','pct_multiple_line']], on='MultipleLines', how='inner')

In [ ]:
# continue with the others categorical columns

freq_is = df['InternetService'].value_counts().reset_index()
freq_is.rename(columns={'index': 'InternetService', 'InternetService': 'freq_internet_service'}, inplace = True)
freq_is['pct_internet_service'] = round((freq_is['freq_internet_service']/freq_is['freq_internet_service'].sum())*100,2)
df = df.merge(freq_is[['InternetService','pct_internet_service']], on='InternetService', how='inner')

freq_con = df['Contract'].value_counts().reset_index()
freq_con.rename(columns={'index': 'Contract', 'Contract': 'freq_contract'}, inplace = True)
freq_con['pct_contract'] = round((freq_con['freq_contract']/freq_con['freq_contract'].sum())*100,2)
df = df.merge(freq_con[['Contract','pct_contract']], on='Contract', how='inner')

freq_pm = df['PaymentMethod'].value_counts().reset_index()
freq_pm.rename(columns={'index': 'PaymentMethod', 'PaymentMethod': 'freq_payment_method'}, inplace = True)
freq_pm['pct_payment_method'] = round((freq_pm['freq_payment_method']/freq_pm['freq_payment_method'].sum())*100,2)
df = df.merge(freq_pm[['PaymentMethod','pct_payment_method']], on='PaymentMethod', how='inner')

df.head(3)

,customerID,gender,SeniorCitizen,tenure,MultipleLines,InternetService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,...,Device Protection,Tech Support,Streaming TV,Streaming Movies,Paperless Billing,Churns,pct_multiple_line,pct_internet_service,pct_contract,pct_payment_method
0,7590-VHVEG,0,0,1,No phone service,DSL,Month-to-month,Electronic check,29.85,29.85,...,0,0,0,0,1,0,9.67,34.36,55.11,33.63
1,8779-QRDMV,1,1,1,No phone service,DSL,Month-to-month,Electronic check,39.65,39.65,...,1,0,0,1,1,1,9.67,34.36,55.11,33.63
2,8665-UTDHZ,1,0,1,No phone service,DSL,Month-to-month,Electronic check,30.20,30.20,...,0,0,0,0,0,1,9.67,34.36,55.11,33.63


In [ ]:
# drop the unused columns

df.drop(['customerID', 'MultipleLines', 'InternetService', 'Contract', 'PaymentMethod'], axis = 'columns', inplace = True)
df.head(3)

,gender,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Partners,Dependentss,Phone Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Paperless Billing,Churns,pct_multiple_line,pct_internet_service,pct_contract,pct_payment_method
0,0,0,1,29.85,29.85,1,0,0,0,1,0,0,0,0,1,0,9.67,34.36,55.11,33.63
1,1,1,1,39.65,39.65,0,0,0,0,0,1,0,0,1,1,1,9.67,34.36,55.11,33.63
2,1,0,1,30.20,30.20,1,1,0,0,1,0,0,0,0,0,1,9.67,34.36,55.11,33.63


In [ ]:
# re-order the columns so Churns column will be the last

df = df[['gender', 'SeniorCitizen','tenure','MonthlyCharges','TotalCharges','Partners','Dependentss','Phone Service','Online Security','Online Backup','Device Protection','Tech Support','Streaming TV','Streaming Movies','Paperless Billing','pct_multiple_line','pct_internet_service','pct_contract','pct_payment_method', 'Churns']]
df.head(3)

,gender,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Partners,Dependentss,Phone Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Paperless Billing,pct_multiple_line,pct_internet_service,pct_contract,pct_payment_method,Churns
0,0,0,1,29.85,29.85,1,0,0,0,1,0,0,0,0,1,9.67,34.36,55.11,33.63,0
1,1,1,1,39.65,39.65,0,0,0,0,0,1,0,0,1,1,9.67,34.36,55.11,33.63,1
2,1,0,1,30.20,30.20,1,1,0,0,1,0,0,0,0,0,9.67,34.36,55.11,33.63,1


In [ ]:
# split train test
from sklearn.model_selection import train_test_split

x = df.drop(['Churns'], axis = 1)
y = df[['Churns']]

# x -> feature
# y -> target

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 42)

In [ ]:
df['Churns'].value_counts()

0    5163
1    1869
Name: Churns, dtype: int64

In [ ]:
df['Churns'].value_counts(normalize = True)*100

0    73.421502
1    26.578498
Name: Churns, dtype: float64

Our dataset is imbalance and actually it's fine. We will continue with modelling

## Modeling (Gunakan lebih min 2 model dan bandingkan hasil evaluasinya)

Bebas menggunakan model, mau menggunakan decision tree, random forest, xgboost, dll juga boleh<br><br>
silahkan berekspresi :)

In [ ]:
# Decision Tree

# fit train
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dt.fit(x_train, y_train)

DecisionTreeClassifier()

In [ ]:
# predict
y_pred_dt = dt.predict(x_test)
y_pred_dt

array([0, 0, 0, ..., 1, 0, 0])

In [ ]:
# calculate AUC
print('AUC', roc_auc_score(y_test, y_pred_dt))

AUC 0.6568467134487863


In [ ]:
# calculate F1 Score
print('F1 Score', f1_score(y_test, y_pred_dt))

F1 Score 0.505091649694501


In [ ]:
# calculate precision
print('Precision', precision_score(y_test, y_pred_dt))

Precision 0.5040650406504065


In [ ]:
# Random Forest

model = RandomForestClassifier()
model.fit(x_train, y_train)

RandomForestClassifier()

In [ ]:
preds = model.predict(x_test)
preds

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
# calculate AUC
print('AUC', roc_auc_score(y_test, preds))

AUC 0.7012666580827915


In [ ]:
# calculate F1 Score
print('F1 Score', f1_score(y_test, preds))

F1 Score 0.5707602339181286


In [ ]:
# calculate precision
print('Precision', precision_score(y_test, preds))

Precision 0.6684931506849315


What if we try to balance the data?

In [ ]:
from imblearn import over_sampling

sm = over_sampling.SMOTE(random_state = 42)

x = df.drop(['Churns'], axis = 1)
y = df[['Churns']]
x_sm, y_sm = sm.fit_resample(x, y)

print(f'''Shape of x before SMOTE : {x.shape}
Shape of x after SMOTE : {x_sm.shape}''')

print('\nBalance(%):')
y_sm.value_counts(normalize = True)*100

Shape of x before SMOTE : (7032, 19)
Shape of x after SMOTE : (10326, 19)

Balance(%):


Churns
0         50.0
1         50.0
dtype: float64

In [ ]:
# Decision Tree

# fit train
from sklearn.tree import DecisionTreeClassifier

dt2 = DecisionTreeClassifier()
dt2.fit(x_train, y_train)

DecisionTreeClassifier()

In [ ]:
# predict
y_pred_dt2 = dt2.predict(x_test)
y_pred_dt2

array([0, 0, 0, ..., 1, 0, 0])

In [ ]:
# calculate accuracy
print('Akurasi', accuracy_score(y_test, y_pred_dt2))

Akurasi 0.732650739476678


In [ ]:
# Random Forest

model2 = RandomForestClassifier()
model2.fit(x_train, y_train)

RandomForestClassifier()

In [ ]:
preds2 = model2.predict(x_test)
preds2

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
# calculate accuracy
print('Akurasi', accuracy_score(y_test, preds2))

Akurasi 0.7901023890784983


## Evaluation

pilih model yang terbaik performannya kemudian beri pejelasan kenapa model tersebut lebih baik dibandingkan dengan yang lain

From both models (Decision Tree and Random Forest) we know the AUC score, Precision score, F1 score for imbalance data and Accuracy score when the data is balance. The AUC score, F1 Score, and accuracy for Random Forest are higher than Decision Tree. Based on the source that I read, Random Forest model is more complex because it is a development of Decision Tree model. Random forest algorithm avoids and prevents overfitting by using multiple trees. While in Decision Tree somehow there is always a scope for overfitting, caused due to the presence of variance.
So, for this dataset I prefer to use Random Forest as the model proven by its high AUC score, F1 Score, Precision Score and in the balanced data we also get the high Accuracy compared to Decision Tree